<a href="https://colab.research.google.com/github/marianaalvesbr/alura-ai-gemini/blob/main/Assistente_IR_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **DeclaraAI - Seu assistente para declaração do importo de renda**

Fazer uma explicação depoos



1 - Instalando as bibliotecas e instanciando as funcionalidades

In [38]:
#---------------------------------------------------------------#
# Configurações Gerais
#---------------------------------------------------------------#

#Iniciando a instalação da bibliote de GenAI do Gemina
%pip -q install google-genai

# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

# Importar biblioteca para ler PDF
!pip install PyPDF2 # Instala a biblioteca se ainda não tiver

# Configura a API Key do Google Gemini
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini
from google import genai
client = genai.Client()
MODEL_ID_2 = "gemini-2.0-flash"
MODEL_ID_25 = "gemini-2.5-flash-preview-04-17"

#Importanto funcionalidades que seram uteis para formatação de texto, criação de agent e runner para rodar os agentes
from IPython.display import HTML, Markdown
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

import PyPDF2

# Permitir upload de arquivo no Colab
from google.colab import files


2 - Agora vou criar algumas funções úteis

In [39]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response


# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


# Função para ler texto de um PDF
def read_pdf(filepath):
    text = ""
    try:
        with open(filepath, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            num_pages = len(reader.pages)
            for page_num in range(num_pages):
                page = reader.pages[page_num]
                text += page.extract_text()
    except Exception as e:
        print(f"Erro ao ler o PDF: {e}")
        text = None # Indica que houve um erro
    return text

# --- Função para limpeza dos outputs dos agentes ---
def clean_agent_output(text):
    # Substitua caracteres problemáticos ou padrões indesejados
    text = text.replace('∗', '') # Exemplo: remover o caractere estrela
    text = text.replace('\n ', '\n') # Exemplo: remover espaços no início de novas linhas (se for um problema)
    # Adicione mais substituições conforme necessário para limpar o texto
    return text

3 - Criando os agentes

In [40]:
###################################################
# --- Agente 1: Buscando informações sobre IR --- #
###################################################

def agente_buscador(ano):
  buscador = Agent(
      name="agente_buscador",
      model=MODEL_ID_2,
      description="Agente que busca informações sobre o imposto de renda do ano incluído pelo usuário",
      tools=[google_search],
      instruction="""
      Você é um agente de pesquisa sobre imposto de renda no Brasil. Sua tarefa é usar a ferramenta de busca do google (google_search)
      para recuperar as últimas informações pertinentes para a declaração do Imposto de Renda com base no ano informado abaixo.

      **Instruções de Formatação da Resposta:**
      - Apresente as informações de forma clara e organizada, usando **Markdown**.
      - Use **títulos e subtítulos** (##, ###) para seções principais (Ex: "Principais Regras", "Prazos", "Novidades").
      - Use **listas com marcadores** (-) para apresentar itens (Ex: Critérios de obrigatoriedade).
      - Use **negrito** (**) para destacar termos importantes (Ex: **Prazo Final**, **Rendimentos Tributáveis**).
      - Evite usar caracteres especiais ou formatação complexa que possa não ser renderizada corretamente.
      - Seja conciso e vá direto ao ponto.
      """
  )

  # Executa o agente
  entrada_do_agente_buscador = f"Ano: {ano}"
  informacoes_buscadas = call_agent(buscador, entrada_do_agente_buscador)
  return informacoes_buscadas





In [48]:
######################################################################
# --- Agente 2: Buscando analisa informações sobre os rendimento --- #
######################################################################

def agente_analisador_rendimentos(ano, pdf_text):
  analisador = Agent(
    name="agente_analisador_rendimentos",
    model=MODEL_ID_25, # Use um modelo mais capaz se possível
    description="Agente que analisa texto de documentos de rendimento e identifica informações relevantes para declaração de imposto de renda.",
    instruction="""
    Você é um assistente especializado em declaração de imposto de renda. Sua tarefa é analisar o texto de documentos de rendimento fornecidos.
    Identifique e liste as seguintes informações, se encontrá-las:
    - Fontes pagadoras (nomes de empresas, bancos, etc.)
    - Tipos de rendimento (salário, aluguéis, aposentadoria, etc.)
    - Valores dos rendimentos
    - Valores de impostos retidos na fonte
    - Outras informações que considere relevantes para a declaração do imposto de renda.
    - Informe o usuário onde no imposto de renda ele deve declarar essas informações

    Liste as informações de rendimento de forma clara, uma informação por linha, sem usar formatação de tabela Markdown.
    """
  )

  # Instanciar o agente analisador
  entrada_do_agente_analisador = f"Ano de refêrencia: {ano}. \n Analise o seguinte texto de um documento de rendimento e extraia as informações relevantes para a declaração de imposto de renda:\n\n{pdf_text}"

  #executa o agente
  analisa_rendimentos = call_agent(analisador, entrada_do_agente_analisador)
  return analisa_rendimentos

In [47]:
#######################################################################
# --- Agente 3: Buscando analisar informações sobre notas fiscais --- #
#######################################################################

def agente_analisador_notas(ano, pdf_text_nota):
  analisador_notas = Agent(
    name="agente_analisador_notas",
    model=MODEL_ID_25, # Use um modelo mais capaz se possível
    description="Agente que analisa texto de documentos de notas fiscais e identifica informações relevantes para declaração de imposto de renda.",
    instruction="""
    Você é um assistente especializado em declaração de imposto de renda. Sua tarefa é analisar o texto de documentos de notas fiscais fornecidos.
    Identifique e liste as seguintes informações, se encontrá-las:
    - Identifique o tipo de nota fiscal, como: serviços de saúde (médico, dentista, psicólogo, etc.), educação (escola, faculdade, cursos),
    ou outros serviços dedutíveis permitidos pela legislação do Imposto de Renda de 2025.
    - Extrair informações do prestador do serviço, como: Nome completo ou razão socual, CPF ou CNPJ, Endereço e outras informações que considerar
    pertinente para facilitar o usuário na declaração
    - Verificar data de emissão e alertar o usuário caso não seja do ano que está declarando.
    - Extrair valores pagos
    - Extrair impostos retidos na fonte
    - Informe a descrição do serviço
    - Outras informações que considere relevantes para a declaração do imposto de renda.
    - Informe o usuário onde no imposto de renda ele deve declarar essas informações

    Apresente um resumo das informações extraídas e diga se ela é provavelmente dedutível, listando os campos que o usuário
    precisará preencher na declaração, sem usar formatação de tabela Markdown.
    """
  )

  # Instanciar o agente analisador
  entrada_do_agente_analisador_notas = f"Ano de refêrencia: {ano}. \n Analise o seguinte texto de um documento de rendimento e extraia as informações relevantes para a declaração de imposto de renda:\n\n{pdf_text_nota}"

  #executa o agente
  analisa_notas = call_agent(analisador_notas, entrada_do_agente_analisador_notas)
  return analisa_notas


In [46]:
#######################################################################
# --- Agente 4: Revisor  --- #
#######################################################################

def agente_revisor(resumo_geral, analise_rendimentos, analise_notas):
  revisor = Agent(
    name="agente_revisor",
    model=MODEL_ID_25, # Use um modelo mais capaz se possível
    description="Agente que analisa texto de documentos de notas fiscais e identifica informações relevantes para declaração de imposto de renda.",
    instruction="""
    Você é um assistente especializado em declaração de imposto de renda. Sua tarefa agora é analisar todos os outputs recebidos, revisar,
    identificar possíveis erros ou inconsistências e organizar as informações.

    - Informe o usuário onde ele deve declarar cada um dos itens analisados
    - Dê dicas consolidadas do que fazer e não fazer na declaração

    Apresente um resumo das informações listando ao usuário todas informações que
    precisará preencher na declaração, sem usar formatação de tabela Markdown.
    """
  )

  # Instanciar o agente analisador
  entrada_do_agente_revisor = f"Ano de refêrencia: {ano}. \n Analise geral dos outputs e consolidação da sua declaração com base nas analises anteriores: \n\n{analise_rendimentos} e {analise_notas}"

  #executa o agente
  declaracao_revisada = call_agent(revisor, entrada_do_agente_revisor)
  return declaracao_revisada


4 - Aplicação principal

In [ ]:
#Aplicação Principal

print("🚀 Iniciando o Assistente de Declaração do Imposto de Renda - DeclaraAI 🚀")

# --- Obter o Ano de referência do Usuário ---
ano = input("❓ Por favor, digite o ano que gostaria de consultar a declaração do imposto de renda: ")

# Inserir lógica do sistema de agentes ################################################
if not ano:
  print("Você esqueceu de digitar o ano")
else:
  print(f"Maravilha! Vamos buscar informações principais para sua declaração do imposto de renda de: {ano}. \n\n")
  print("\n-- AGUARDE (Buscando informações gerais sobre IR para o ano) --\n")

  # --- Chamando o primeiro agente que vai trazer as informações básicas sobre o imposto
  informacoes_buscadas = agente_buscador(ano)

  #Fazendo uma limpeza do output
  cleaned_informacoes_buscadas = clean_agent_output(informacoes_buscadas)


  print("\n-- Resultado do Agente com Informações de IR --\n")
  display(to_markdown(cleaned_informacoes_buscadas))
  print("---------------------------------------------\n\n")

  ##########################################################################
  # --- Pedindo ao usuário para fazer upload dos arquivos de rendimento ---#
  ##########################################################################

  print("\n📄 Agora, por favor, faça o upload dos seus arquivos de informe de rendimentos em formato PDF.")
  print("\n👉 Clique no botão 'Escolher arquivos' que aparecerá abaixo para selecionar os documentos.")
  print("\n✅ Você pode enviar um ou mais arquivos de uma vez.")

  #solicitando o arquivo
  upload_rendimentos = files.upload()

  if not upload_rendimentos:
    print("❌ Nenhum arquivo foi enviado. Por favor, tente novamente.")
  else:
    # --- Informar nome dos arquivos ---
    print("Maravilha! Recebemos seus arquivos.")

    # informando o nome dos arquivos enviados
    print("\nEsses foram os arquivos enviados e que vamos avaliar")
    for filename in upload_rendimentos.keys():
        print(f"- {filename}")
    print("-" * 30) # Linha separadora para clareza

    # Mensagem de espera mais específica
    print("\n-- AGUARDE (Analisando rendimentos dos arquivos enviados) --\n")

    # Lista para armazenar os resultados da análise de cada arquivo
    resultados_analise_arquivos = []

    for filename in upload_rendimentos.keys(): # Itera sobre todos os arquivos enviados
        print(f"Processando arquivo: {filename}...") # Informa qual arquivo está processando
        pdf_text = read_pdf(filename) # Lê o texto deste arquivo específico

        if pdf_text:
            # Chama o agente analisador para CADA arquivo
            print(f"  ✅ Texto extraído. Enviando para análise do agente...")
            analise_individual = agente_analisador_rendimentos(ano, pdf_text)

            # Armazenar o resultado
            resultados_analise_arquivos.append({
                "arquivo": filename,
                "analise": analise_individual
            })

        else:
            print(f"  ❌ Não foi possível extrair texto do arquivo: {filename}. Pulando análise deste arquivo.")


    # Exibir todos os resultados acumulados (ou resumir)
    if resultados_analise_arquivos:
        print("\n--- RESUMO DA ANÁLISE DE TODOS OS ARQUIVOS DE RENDIMENTO ENVIADOS ---\n")
        # Concatena os resultados para exibição
        resumo_total = ""
        for resultado in resultados_analise_arquivos:
            resumo_total += f"--- Análise para {resultado['arquivo']} ---\n"
            resumo_total += resultado['analise'] + "\n\n" # Adiciona a análise individual
        display(to_markdown(resumo_total)) # Exibe o resumo
        print("----------------------------------------------------------------------\n\n")
    else:
      # Esta mensagem só aparecerá se o usuário UPOU arquivos, mas NENHUM deles pôde ser lido.
      print("\n❌ Nenhum dos arquivos enviados pôde ser processado ou analisado com sucesso.")
      # --- Informar nome dos arquivos ---
      print("Maravilha! Recebemos seus arquivos.")

      # informando o nome dos arquivos enviados
      print("\nEsses foram os arquivos enviados e que vamos avaliar")

      for filename in upload_rendimentos.keys():
          print(f"- {filename}")
      print("-" * 30) # Linha separadora para clareza

      # Mensagem de espera mais específica
      print("\n-- AGUARDE (Analisando rendimentos dos arquivos enviados) --\n")

      # Lista para armazenar os resultados da análise de cada arquivo
      resultados_analise_arquivos = []

      for filename in upload_rendimentos.keys(): # Itera sobre todos os arquivos enviados
          print(f"Processando arquivo: {filename}...") # Informa qual arquivo está processando
          pdf_text = read_pdf(filename) # Lê o texto deste arquivo específico

          if pdf_text:
              # Chama o agente analisador para CADA arquivo
              print(f"  ✅ Texto extraído. Enviando para análise do agente...")
              analise_individual = agente_analisador_rendimentos(ano, pdf_text)

              # Armazenar o resultado
              resultados_analise_arquivos.append({
                  "arquivo": filename,
                  "analise": analise_individual
              })

          else:
              print(f"  ❌ Não foi possível extrair texto do arquivo: {filename}. Pulando análise deste arquivo.")


      # Exibir todos os resultados acumulados (ou resumir)
      if resultados_analise_arquivos:
          print("\n--- RESUMO DA ANÁLISE DE TODOS OS ARQUIVOS DE RENDIMENTO ENVIADOS ---\n")
          # Concatena os resultados para exibição
          resumo_total = ""
          for resultado in resultados_analise_arquivos:
              resumo_total += f"--- Análise para {resultado['arquivo']} ---\n"
              resumo_total += resultado['analise'] + "\n\n" # Adiciona a análise individual
          display(to_markdown(resumo_total)) # Exibe o resumo
          print("----------------------------------------------------------------------\n\n")
      else:
          # Esta mensagem só aparecerá se o usuário UPOU arquivos, mas NENHUM deles pôde ser lido.
          print("\n❌ Nenhum dos arquivos enviados pôde ser processado ou analisado com sucesso.")

  ######################################################################
  # --- Pedindo ao usuário para fazer upload dos arquivos de notas --- #
  ######################################################################

  print("\n📄 Agora, por favor, faça o upload dos seus arquivos de notas fiscais em formato PDF.")
  print("\n👉 Clique no botão 'Escolher arquivos' que aparecerá abaixo para selecionar os documentos.")
  print("\n✅ Você pode enviar um ou mais arquivos de uma vez.")

  #solicitando o arquivo
  upload_notas = files.upload()

  if not upload_notas:
    print("❌ Nenhum arquivo de nota fiscal foi enviado. Por favor, tente novamente.")
  else:
    # --- Informar nome dos arquivos de notas fiscais ---
    print("Maravilha! Recebemos seus arquivos de notas fiscais.")

    # informando o nome dos arquivos enviados
    print("\nEsses foram os arquivos de notas fiscais enviados e que vamos avaliar")
    for filename in upload_notas.keys(): # <--- Usar upload_notas aqui
        print(f"- {filename}")
    print("-" * 30) # Linha separadora para clareza

    # Mensagem de espera mais específica para notas fiscais
    print("\n-- AGUARDE (Analisando notas fiscais dos arquivos enviados) --\n")

    # Lista para armazenar os resultados da análise de cada arquivo de nota
    resultados_analise_notas = [] # <--- Nova lista para resultados de notas

    for filename in upload_notas.keys(): # <--- Iterar sobre upload_notas
        print(f"Processando arquivo de nota: {filename}...") # Mensagem específica
        pdf_text_nota = read_pdf(filename) # <--- Variável para texto da nota

        if pdf_text_nota: # <--- Usar texto da nota
            # Chama o agente analisador para CADA arquivo de nota
            print(f"  ✅ Texto de nota fiscal extraído. Enviando para análise do agente de notas fiscais...")
            # *** CHAMAR O TERCEIRO AGENTE AQUI ***
            analise_individual_nota = agente_analisador_notas(ano, pdf_text_nota) # <--- Chamar o agente_analisador_notas

            # Armazenar o resultado
            resultados_analise_notas.append({
                "arquivo": filename,
                "analise": analise_individual_nota
            })

        else:
            print(f"  ❌ Não foi possível extrair texto do arquivo de nota fiscal: {filename}. Pulando análise deste arquivo.")


    # Exibir todos os resultados acumulados da análise de notas
    if resultados_analise_notas:
        print("\n--- RESUMO DA ANÁLISE DE TODOS OS ARQUIVOS DE NOTAS FISCAIS ENVIADOS ---\n")
        # Concatena os resultados para exibição
        resumo_total_notas = "" # <--- Nova variável para resumo de notas
        for resultado_nota in resultados_analise_notas: # <--- Iterar sobre resultados_analise_notas
            resumo_total_notas += f"--- Análise para {resultado_nota['arquivo']} ---\n"
            resumo_total_notas += resultado_nota['analise'] + "\n\n" # Adiciona a análise individual
        display(to_markdown(resumo_total_notas)) # <--- Exibir resumo de notas
        print("--------------------------------------------------------------------------\n\n")
    else:
      # Esta mensagem só aparecerá se o usuário UPOU arquivos de nota, mas NENHUM deles pôde ser lido.
      print("\n❌ Nenhum dos arquivos de notas fiscais enviados pôde ser processado ou analisado com sucesso.")

  print("Aguarde enquanto finalizamos a revis±ao geral de tudo que você viu para que possamos dar dicas para seua declaração...")
  informacao_final = agente_revisor(cleaned_informacoes_buscadas, resumo_total, resumo_total_notas)
  print("\n-- Resultado Final do seu Assitente com dicas e informações consolidadas sobre seu imposto de renda --\n")
  display(to_markdown(informacao_final))
  print("---------------------------------------------\n\n")
